# Test: tSZxCIB

Here we compute the difference between the tSZxCIB template used in Choi et al 2020 computed using fgspectra and the [Fortran](https://lambda.gsfc.nasa.gov/product/act/act_dr4_likelihood_get.cfm) implementation.

In [1]:
%matplotlib inline 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from fgspectra import cross as fgc
from fgspectra import frequency as fgf
from fgspectra import power as fgp

Define parameters needed to compute the model

In [4]:
fg_params = {
            'xi': 0.1998220E+00,
            'a_tSZ': 0.5806115E+01,
            'a_c': 0.3648102E+01,
            'T_d': 9.7,
            'beta_p': 0.2447908E+01,
            }

freqs = np.array([98,150])
nu_0 = 150.
ell_0 = 3000
ell = np.arange(2,7926)


tSZ_and_CIB = fgc.SZxCIB_Choi2020()

Now we can compute the sum of tSZ, CIB and tSZxCIB using fgspectra

In [5]:
test = tSZ_and_CIB(
                {'kwseq': (
                    {'nu':freqs, 'nu_0': nu_0},
                    {'nu': freqs, 'nu_0': nu_0, 'temp': fg_params['T_d'], 'beta': fg_params['beta_p']} 
                    )},
                {'kwseq': ( 
                    {'ell':ell, 'ell_0': ell_0, 
                     'amp': fg_params['a_tSZ']},
                    {'ell':ell, 'ell_0': ell_0, 'amp': fg_params['a_c']},
                    {'ell':ell, 'ell_0': ell_0, 
                     'amp': - fg_params['xi'] * np.sqrt(fg_params['a_tSZ'] * fg_params['a_c'])}
                    )}) 

test_dict = {}

for c1, f1 in enumerate(freqs):
        for c2, f2 in enumerate(freqs):
            test_dict[f1, f2] = test[c1, c2]
            

The file 'sum_CIB_tSZ_tSZxCIB_fortran_routine.txt' contains the sum of tSZ, CIB and tSZxCIB computed using the implementation in the likelihood fortran code. You check the numerical differences:

In [6]:
tSZ_and_CIB_f90Routine = np.loadtxt('sum_CIB_tSZ_tSZxCIB_fortran_routine.txt',unpack=True)

reldiff_98x98 = np.abs(test_dict[98,98]-tSZ_and_CIB_f90Routine[0,:])/tSZ_and_CIB_f90Routine[0,:]

In [7]:
reldiff_98x98.max()

6.856095485299012e-16